In [23]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train="False",
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data,batch_size=64)
test_dataloader = DataLoader(training_data,batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [24]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [25]:
loss_fn = nn.CrossEntropyLoss()

In [26]:
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [27]:
def train_loop(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)

    model.train()
    for batch,(X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0 :
            loss,current = loss.item(),batch*batch_size + len(X)
            print(loss,current,size)

def test_loop(data_loader,model,loss_fn):
    model.eval()
    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    test_loss,correct = 0,0

    with torch.no_grad():
        for X,y in data_loader:
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(100*correct,test_loss) 

In [28]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
2.2993292808532715 64 60000
2.292097330093384 6464 60000
2.2710788249969482 12864 60000
2.2636895179748535 19264 60000
2.25504732131958 25664 60000
2.2218029499053955 32064 60000
2.2274434566497803 38464 60000
2.1868531703948975 44864 60000
2.1874310970306396 51264 60000
2.1600677967071533 57664 60000
45.50333333333334 2.151821105210766
Epoch 2
-------------------------------
2.1618759632110596 64 60000
2.152878761291504 6464 60000
2.0928914546966553 12864 60000
2.1105809211730957 19264 60000
2.06526780128479 25664 60000
2.000793695449829 32064 60000
2.0263566970825195 38464 60000
1.9403131008148193 44864 60000
1.9464349746704102 51264 60000
1.8791674375534058 57664 60000
63.79666666666667 1.8690315027480948
Epoch 3
-------------------------------
1.9079623222351074 64 60000
1.870216965675354 6464 60000
1.7549924850463867 12864 60000
1.7996983528137207 19264 60000
1.6944106817245483 25664 60000
1.6455374956130981 32064 60000
1.667769193649292 384